In [3]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ultralytics-8.0.117-py3-none-any.whl (599 kB)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
!cp "/content/drive/MyDrive/datasets/hgv_dataset2.zip" "hgv_dataset2.zip"

In [ ]:
!unzip hgv_dataset2.zip -d ./hgv_dataset1

In [11]:
import os
import shutil
import numpy as np

# Define source and destination directories
src_dir = './hgv_dataset1'

# Get list of all files in source directory
files = os.listdir(src_dir)

# Filter the list to only include JPG images
jpg_files = [f for f in files if f.endswith('.png')]
txt_files = [f for f in files if f.endswith('.txt')]

# Sort the list of JPG files by name
jpg_files.sort()
txt_files.sort()

# Generate a randomly permuted sequence of indices
indices = np.random.permutation(len(jpg_files))

# Use the indices to shuffle the sorted list
shuffled_jpg = [jpg_files[i] for i in indices]
shuffled_txt = [txt_files[i] for i in indices]
print(shuffled_jpg)
print(shuffled_txt)

['2023-05-19_20-14-03_clip_2_00007.png', '2023-05-06_17-34-59_clip_3_00012.png', '2023-05-19_22-11-54_clip_2_00000.png', '2023-05-19_20-06-30_clip_6_00005.png', '2023-05-19_21-30-33_clip_3_00001.png', '2023-05-19_21-30-33_clip_3_00000.png', '2023-05-06_17-36-50_clip_1_00007.png', '2023-05-19_22-14-10_clip_2_00001.png', '2023-05-19_21-49-37_clip_2_00014.png', '2023-05-06_17-27-18_clip_1_00042.png', '2023-05-19_22-13-42_clip_1_00003.png', '2023-05-19_22-14-10_clip_2_00000.png', '2023-05-19_21-32-50_clip_1_00004.png', '2023-05-19_20-01-53_clip_2_00001.png', '2023-05-19_21-20-15_clip_2_00009.png', '2023-05-19_22-11-54_clip_2_00006.png', '2023-05-19_20-09-57_clip_2_00006.png', '2023-05-06_17-27-03_clip_2_00046.png', '2023-05-19_21-15-05_clip_5_00001.png', '2023-05-19_20-06-30_clip_2_00006.png', '2023-05-06_17-40-17_clip_1_00000.png', '2023-05-19_21-51-12_clip_1_00060.png', '2023-05-19_22-18-13_clip_3_00010.png', '2023-05-19_22-11-26_clip_1_00005.png', '2023-05-06_17-35-54_clip_1_00004.png',

In [12]:
file_dirs = [
    '/content/dataset/images/train',
    '/content/dataset/labels/train',
    '/content/dataset/images/val',
    '/content/dataset/labels/val'
]

# Calculate 80% of the length of the filtered list
train_n = len(shuffled_jpg) * 80 // 100
val_n = len(shuffled_jpg) * 20 // 100

# Select the top 80% of the shuffled JPG images
file_lists = [
    shuffled_jpg[:train_n],
    shuffled_txt[:train_n],
    shuffled_jpg[-val_n:],
    shuffled_txt[-val_n:]
]

# Iterate over selected files and copy them to destination directory
for dir_, file_list in zip(file_dirs, file_lists):
  for file_ in file_list:
    src_path = os.path.join(src_dir, file_)
    dst_path = os.path.join(dir_, file_)
    shutil.copy(src_path, dst_path)

In [13]:
import os
print(len(os.listdir("/content/dataset/images/train")))
print(len(os.listdir("/content/dataset/images/val")))
print(len(os.listdir("/content/dataset/labels/train")))
print(len(os.listdir("/content/dataset/labels/val")))


1000
250
1000
250


In [ ]:
# run for both labels/val and labels/train
for file in os.listdir("/content/dataset/labels/val"):
  path = "/content/dataset/labels/val/" + file

  annotations = []
  with open(path, 'r') as file:
    lines = file.readlines()

  for line in lines:
    x, y, w, h = line.split(" ")[1:]
    annotations.append(f"0 {float(x)/1920} {float(y)/1080} {float(w)/1920} {float(h)/1080}")
    print(line)
  annotations = "\n".join(annotations)
  print(annotations)

  with open(path, 'w') as file:
    print(f"annotattions {annotations}")
    file.write(annotations)

In [16]:
with open("/content/dataset/labels/train/" + os.listdir("/content/dataset/labels/train")[0], 'r') as file:
  print(file.readlines())

['0 0.3575520833333333 0.1648148148148148 0.0890625 0.06481481481481481']


In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8x.yaml').load('yolov8x.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
model.train(data='/content/dataset.yaml', epochs=50, imgsz=640, workers=0)


                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640

In [20]:
!cp "/content/runs/detect/train/weights/best.pt" "best.pt"

In [21]:
from pathlib import Path
from ultralytics import YOLO
img_in  = str(Path("/content/2023-05-19_22-15-30.mp4"))
model   = YOLO("best.pt")
results = model(img_in, save=True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 68.6ms
video 1/1 (2/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 52.7ms
video 1/1 (3/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 52.8ms
video 1/1 (4/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 52.6ms
video 1/1 (5/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 51.2ms
video 1/1 (6/285) /content/2023-05-19_22-15-30.mp4: 384x640 2 hgvs, 33.3ms
video 1/1

In [ ]:
!rm -rf /content/hgvDataset

In [ ]:
import os

# Define the source directory where the files are located
src_dir = '/content/hgvDataset'

# Get a list of all the image files in the directory
img_files = [f for f in os.listdir(src_dir) if f.endswith('.jpg')]

# Iterate over the image files and check if the corresponding text file has data
for img_file in img_files:
    img_path = os.path.join(src_dir, img_file)
    txt_file = os.path.splitext(img_file)[0] + '.txt'
    txt_path = os.path.join(src_dir, txt_file)
    if os.path.exists(txt_path) and os.path.getsize(txt_path) == 0:
        os.remove(img_path)
        os.remove(txt_path)
        print(f'Removing empty text file and image: {txt_file}, {img_file}')


In [ ]:
!zip -r /content/nano-hgv.zip /content/runs/detect/train6/

In [ ]:
!cp "/content/runs/detect/train6/weights/best.pt" "best.pt"